In [8]:
import os
import gc
import shutil
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from microns_phase3 import nda
from ccmodels.preprocessing.connectomics import client_version, subset_v1l234
from ccmodels.preprocessing.selectivity import orientation_extractor, von_mises, von_mises_single, is_selective, cell_area_identifiers


In [3]:
#Identify desired neurons, in this case v1 neurons from L234
client = client_version(661)
area_v1_neurons = cell_area_identifiers('V1')
v1l234_neur = subset_v1l234(client, table_name = 'coregistration_manual_v3', area_df = area_v1_neurons)
v1l234_neur = v1l234_neur[v1l234_neur['pt_root_id'] != 0]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [13]:
v1l234_neur.groupby(['session', 'scan_idx']).count()

pt_root_id   id  unit_id  pt_position  brain_area  \
session scan_idx                                                      
4       7                538  538      538          538         538   
5       3                101  101      101          101         101   
        6                380  380      380          380         380   
        7                436  436      436          436         436   
6       2                501  501      501          501         501   
        4                511  511      511          511         511   
        6                444  444      444          444         444   
        7                489  489      489          489         489   
7       3                340  340      340          340         340   
        4                581  581      581          581         581   
        5                317  317      317          317         317   
8       5                633  633      633          633         633   
        7                  3    3        3            3           3   
9       3                781  781      781          781         781   
        4                737  737      737          737         737   
        6                300  300      300          300         300   

                  pial_distances  cortex_layer  
session scan_idx                                
4       7                    538           538  
5       3                    101           101  
        6                    380           380  
        7                    436           436  
6       2                    501           501  
        4                    511           511  
        6                    444           444  
        7                    489           489  
7       3                    340           340  
        4                    581           581  
        5                    317           317  
8       5                    633           633  
        7                      3             3  
9       3                    781           781  
        4                    737           737  
        6                    300           300

In [18]:
t = pd.read_pickle('/workspaces/microns_phase3_nda/data/in_processing/orientation_fits.pkl')

In [11]:
nda.Scan().fetch(format = 'frame')

nframes  nfields      fps
session scan_idx                           
4       7           40000        8  6.30090
        9           35112        8  6.30090
        10          40000        8  6.30090
5       3           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
6       2           40000        8  6.30090
        4           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
7       3           40000        8  6.30090
        4           40000        8  6.30090
        5           40000        8  6.30090
8       5           40000        8  6.30090
        7           40000        8  6.30090
        9           40000        8  6.30090
9       3           50000        6  8.61754
        4           50000        6  8.61754
        6           57000        4  9.62246

In [15]:
ors_all = pd.read_pickle(f"../data/in_processing/orientation_fits/{os.listdir('../data/in_processing/orientation_fits')[0]}")

#Loading the rest iteratively and concatenating
for file in tqdm(os.listdir('../data/in_processing/orientation_fits/')[1:], desc='Aggregating session, scan files'):
    if file!='.DS_Store':
        cont_df = pd.read_pickle(f'../data/in_processing/orientation_fits/{file}')
        ors_all = pd.concat([ors_all, cont_df], axis = 0)
    else:
        continue

#Saving the data
ors_all.to_pickle('../data/in_processing/orientation_fits.pkl')

Aggregating session, scan files:   0%|          | 0/15 [00:00<?, ?it/s]

In [16]:
shutil.rmtree('../data/in_processing/orientation_fits')